In [13]:
def calculate_mask_and_columns(tensor, V, N, M):
    rows, cols = tensor.shape
    mask = torch.zeros_like(tensor, dtype=torch.int32)  # 使用 int32 类型
    columns = torch.zeros((rows // V, cols // M * 4), dtype=torch.int32)  # 存储每个块中非零元素的列索引

    # 遍历每个 VxM 的块
    for row_block in range(0, rows, V):
        for col_block in range(0, cols, M):
            block = tensor[row_block:row_block + V, col_block:col_block + M]

            # 确定块中所有非零元素的列位置
            nonzero_cols = block.nonzero(as_tuple=False)[:, 1]  # 获取非零元素的列索引

            if len(nonzero_cols) > 0:
                # 从中提取前四个不重复的列索引
                unique_cols = torch.unique(nonzero_cols)
                num_unique = min(len(unique_cols), 4)  # 最多取四列
                
                # 更新 columns 矩阵
                columns[row_block // V, (col_block // M) * 4: (col_block // M) * 4 + num_unique] = unique_cols[:num_unique]
                
                # 设置 mask
                for idx in range(V):  # 对于块中的每一行
                    row_nonzero_cols = block[idx, :].nonzero(as_tuple=False).squeeze()
                    mask[row_block + idx, col_block + row_nonzero_cols] = 1

    return mask, columns

1048576